In [61]:
# Vector3 Class
from dataclasses import dataclass
import math

@dataclass(frozen=True)
class Vector3:
    x: float
    y: float
    z: float

    # Vector addition
    def __add__(self, other: "Vector3") -> "Vector3":
        return Vector3(
            self.x + other.x,
            self.y + other.y,
            self.z + other.z
        )

    # Vector subtraction
    def __sub__(self, other: "Vector3") -> "Vector3":
        return Vector3(
            self.x - other.x,
            self.y - other.y,
            self.z - other.z
        )

    # Scalar multiplication
    def __mul__(self, scalar: float) -> "Vector3":
        return Vector3(
            self.x * scalar,
            self.y * scalar,
            self.z * scalar
        )
    
    # Scalar division
    def __truediv__(self, scalar: float) -> "Vector3":
        return Vector3(
            self.x / scalar,
            self.y / scalar,
            self.z / scalar
        )

    __rmul__ = __mul__
    __rtruediv__ = __truediv__

    # Dot product
    def dot(self, other: "Vector3") -> float:
        return (
            self.x * other.x +
            self.y * other.y +
            self.z * other.z
        )

    # Cross product
    def cross(self, other: "Vector3") -> "Vector3":
        return Vector3(
            self.y * other.z - self.z * other.y,
            self.z * other.x - self.x * other.z,
            self.x * other.y - self.y * other.x
        )

    # Magnitude (length)
    def magnitude(self) -> float:
        return math.sqrt(self.dot(self))

    # Unit vector
    def unit_vector(self) -> "Vector3":
        mag = self.magnitude()
        if mag == 0:
            raise ValueError("Cannot normalize a zero vector")
        return self * (1 / mag)

In [62]:
# Semi Major Axis (a)
def compute_sma(mu:float, energy:float):
    return -mu/(2*energy)


In [63]:
# Eccentricity (e)
import math

def compute_ecc(energy: float, mu: float, h:float):
    return math.sqrt(1+(2*math.pow(h, 2)*energy)/(math.pow(mu, 2)))

In [64]:
# Inclination (i)
import math

def compute_inc(Z_: Vector3, h_: Vector3):
    Z_ = Z_.unit_vector()
    h_ = h_.unit_vector()
    return math.acos(Z_.dot(h_))


In [65]:
# RAAN (capital omega) Ω
import math

def compute_raan(Nx: float, Ny: float):
    return math.atan2(Ny, Nx)

In [66]:
# Argument of Perigeee (omega sub p) ωp
import math

def compute_argp(h_: Vector3, N_: Vector3, B_: Vector3):
    h_ = h_.unit_vector()
    N_ = N_.unit_vector()
    B_ = B_.unit_vector()
    return math.atan2(h_.dot(N_.cross(B_)), N_.dot(B_))



In [67]:
# True Anomaly (nu) ν
import math

def compute_ta(v_: Vector3, r_: Vector3, B_: Vector3):
    cosTa = r_.dot(B_)/(r_.magnitude()*B_.magnitude())
    ta = math.acos(cosTa)
    if r_.dot(v_) < 0:
        ta = 2*math.pi - ta
    return ta

In [68]:
# Period (TP)
import math

def compute_period(a: float, mu: float):
    return 2*math.pi*math.sqrt(math.pow(a, 3)/mu)

In [69]:
# Apogee (r sub a) ra
def compute_apogee(a: float, e: float):
    return a*(1+e)

In [70]:
# Perigee (r sub p) rp
def compute_perigee(a: float, e: float):
    return a*(1-e)

In [71]:
# Energy (xi) ξ
import math

def compute_energy(v: float, r: float, mu: float):
    return math.pow(v, 2)/2 - mu/r

In [72]:
# Angular Momentum Vector (h_)
def compute_angular_momentum_vector(r: Vector3, v: Vector3):
    return r.cross(v)

In [73]:
# RAAN Vector (N_) Unitized
def compute_raan_vector(Z_: Vector3, h_: Vector3):
    Z_ = Z_.unit_vector()
    h_ = h_.unit_vector()
    return (Z_.cross(h_)/(Z_.cross(h_).magnitude())).unit_vector()

In [74]:
# Perigee Vector (B_) Unitized
def compute_perigee_vector(mu: float, r_: Vector3, v_: Vector3, h_: Vector3):
    return v_.cross(h_)-mu*(r_/r_.magnitude()).unit_vector()

In [75]:
# Keplerian Elements Class
import math

class KeplerianElements:
    """
    Computes classical Keplerian orbital elements from
    position and velocity state vectors.
    """

    mu_earth = 3.986004418e14  # m^3 / s^2

    def __init__(self, position: Vector3, velocity: Vector3):
        """
        Parameters
        ----------
        position : Vector3
            Position vector (meters)
        velocity : Vector3
            Velocity vector (m/s)
        """

        mu = self.mu_earth
        k_hat = Vector3(0, 0, 1)

        # Specific orbital energy
        self.xi = compute_energy(velocity.magnitude(), position.magnitude(), mu)

        # Semi-major axis
        self.a = compute_sma(mu, self.xi)

        # Angular momentum vector
        self.h_ = compute_angular_momentum_vector(position, velocity)

        # Eccentricity
        self.ecc = compute_ecc(self.xi, mu, self.h_.magnitude())

        # Inclination
        self.inc = compute_inc(k_hat, self.h_)
        self.inc_deg = math.degrees(self.inc)

        # RAAN
        self.N_Unit = compute_raan_vector(k_hat, self.h_)
        self.raan = compute_raan(self.N_Unit.x, self.N_Unit.y)
        self.raan_deg = math.degrees(self.raan)

        # Argument of perigee
        self.B_Unit = compute_perigee_vector(
            mu,
            position,
            velocity,
            self.h_
        )
        self.argp = compute_argp(self.h_, self.N_Unit, self.B_Unit)
        self.argp_deg = math.degrees(self.argp)

        # True anomaly
        self.ta = compute_ta(velocity, position, self.B_Unit)
        self.ta_deg = math.degrees(self.ta)

        # Orbital period (elliptical only)
        self.period = compute_period(self.a, mu)

        # Apoapsis / periapsis
        self.apogee = compute_apogee(self.a, self.ecc)
        self.perigee = compute_perigee(self.a, self.ecc)


```^ Previous Lessons ^```

# Problem 1:

In [76]:
pos = Vector3(326151.080726, 6077471.251787, 2944583.918767)  # meters
vel = Vector3(-7455.178720, -482.482572, 1910.883434) # m/s

kepler_elements = KeplerianElements(pos, vel)
print("nu (true anomaly) is: " + str(kepler_elements.ta) + " rad")

nu (true anomaly) is: 0.533708002792791 rad


In [77]:
# Eccentric Anomaly (E)
import math

def compute_eccentric_anomaly(nu: float, e: float) -> float:
    return math.asin((math.sin(nu)*math.sqrt(1 - math.pow(e, 2))) / (1 + e*math.cos(nu)))


In [78]:
E_SV = compute_eccentric_anomaly(kepler_elements.ta, kepler_elements.ecc)
print("E_SV (eccentric anomaly) is: " + str(E_SV) + " rad")

E_SV (eccentric anomaly) is: 0.5286424011417852 rad


In [79]:
# Mean Anomaly (M)
import math

def compute_mean_anomaly(E: float, e: float) -> float:
    return E - e*math.sin(E)

In [80]:
M_SV = compute_mean_anomaly(E_SV, kepler_elements.ecc)
print("M0 (mean anomaly) is: " + str(M_SV) + " rad")

M0 (mean anomaly) is: 0.5235987858960514 rad


In [81]:
# Mean Motion (n)
import math

def compute_mean_motion(mu: float, a: float):
    # note a is semi major axis in meters
    return math.sqrt(mu/math.pow(a,3))

##### Time of flight from perigee to nu (true anomaly)
Initially:  
nu (true anomaly) is: 0.533708002792791 rad  
E (eccentric anomaly) is: 0.5286424011417852 rad  
M (mean anomaly) is: 0.5235987858960514 rad  
  
At Perigee:  
tp = 0  
Ep = 0  

General Time of Flight Equation:  
`t-t0 = kTP + (1/n)(E - e * sin(E)) - (1/n) * (E0 - e * sin(E0))`  
- T = Time of last perigee passage  
- t0 = Time from perigee to E0  
- t = Time from perigee to E  
- k = perigee passages between E0 and E  
- k * TP = number of full orbits from epoch  

Therefore  
t0 is 0 and k is 0 and E0 is 0 so  
`t = (1/n)(E - e*sin(E))`

In [82]:
n = compute_mean_motion(kepler_elements.mu_earth, kepler_elements.a)
print("n (mean motion) is: " + str(n) + " rad/sec") 

n (mean motion) is: 0.0011209657051213528 rad/sec


In [83]:
t_delta_perigeee_to_nu = (1/n)*(E_SV-kepler_elements.ecc * math.sin(E_SV))
print("time delta between perigee and nu: " + str(t_delta_perigeee_to_nu) + " sec")

time delta between perigee and nu: 467.0961685124594 sec


##### Time of flight from nu to nu = 65deg
Initially:  
nu (true anomaly) is: 0.533708002792791 rad  
E (eccentric anomaly) is: 0.5286424011417852 rad  
M (mean anomaly) is: 0.5235987858960514 rad  
  
At nu = 65deg:  
calculate E

General Time of Flight Equation:  
`t-t0 = kTP + (1/n)(E - e * sin(E)) - (1/n) * (E0 - e * sin(E0))`  
- T = Time of last perigee passage  
- t0 = Time from perigee to E0  
- t = Time from perigee to E  
- k = perigee passages between E0 and E  
- k * TP = number of full orbits from epoch  

Therefore  
t0 is 0 and k is 0 and E0 is 0.5286424011417852 so  
`t = (1/n)(E_65-e*sin(E65)) - (1/n)*(E0-e*sin(E0))`

In [84]:
def compute_time_delta_after_angle(rad_angle: float, kepler_elements: KeplerianElements) -> tuple[float, float]:
    n = compute_mean_motion(kepler_elements.mu_earth, kepler_elements.a)
    E_SV = compute_eccentric_anomaly(kepler_elements.ta, kepler_elements.ecc)
    E_angle = compute_eccentric_anomaly(rad_angle, kepler_elements.ecc)
    t_delta_nu_to_angle = (1/n)*(E_angle-kepler_elements.ecc*math.sin(E_angle)) - (1/n)*(E_SV-kepler_elements.ecc*math.sin(E_SV))
    #if t is negative, then we add the orbital period to it to get the next time it will hit the angle
    # if t_delta_nu_to_angle < 0:
    #     t_delta_nu_to_angle += 2*math.pi/n  # orbital period
    return E_angle, t_delta_nu_to_angle

In [85]:
E_65, t_delta_nu_to_65 = compute_time_delta_after_angle(math.radians(65), kepler_elements)
print("E_65 is " + str(E_65))
print("time delta between nu and 65 deg: " + str(t_delta_nu_to_65) + " sec")

E_65 is 1.1254198827627566
time delta between nu and 65 deg: 528.8267149213564 sec


##### Verification
Given 528.8267149213564 sec, now I need to verify that after that time, we will be at nu=65 deg

Time to solve for E by Newton-Raphson
`E0 = M` (mean anomaly. Mean Motion is constant (rad/sec), Mean anomaly is not)
M for Newton-Raphson is: `M = n(t-T)` (essentially rad/sec*sec which gives you rad)
remember the formula for `E_k_1=E_k + (M-(E_k-e*sin(E_k)))/(1-e*cos(E_k))`  
Solve for nu after you get E using `cos(nu) = (e-cos(E))/(e*cos(E)-1)`

In [86]:
import math

# compute E based on time delta
M_propagated = M_SV + n*t_delta_nu_to_65
E_k = M_propagated
E_k_1 = 100000000000 # just to make sure our diff is large enough for first iteration
count = 0
while (abs(E_k-E_k_1) != 0):
    E_k = E_k_1
    E_k_1=E_k + (M_propagated-(E_k-kepler_elements.ecc*math.sin(E_k)))/(1-kepler_elements.ecc*math.cos(E_k))
    count = count + 1
    print("Iteration " + str(count) + ": " + str(abs(E_k-E_k_1)))
E_check = E_k_1%(2*math.pi)
print("Eccentric Anomaly Propagated: " + str(E_check) + " rad") # mod by 2pi in case k is greater than 0
print("k: " + str(math.floor((E_k_1-E_SV))/(2*math.pi)))
print("Diff between E_65 and computed E_65 through propagation is: " + str(abs(E_65-E_check)))


Iteration 1: 100372228187.19342
Iteration 2: 375802045.9425468
Iteration 3: 3565896.24297107
Iteration 4: 7950.319210706047
Iteration 5: 11.178083766057082
Iteration 6: 0.11661818978111382
Iteration 7: 5.913950939429036e-05
Iteration 8: 1.5849987988758585e-11
Iteration 9: 0.0
Eccentric Anomaly Propagated: 1.1254198827627564 rad
k: 0.0
Diff between E_65 and computed E_65 through propagation is: 2.220446049250313e-16


In [87]:
import math

# compute true anomaly from E
nu_check_cos = (kepler_elements.ecc-math.cos(E_check))/(kepler_elements.ecc*math.cos(E_check)-1)
nu_check_sin = (math.sin(E_check)*math.sqrt(1-math.pow(kepler_elements.ecc, 2)))/(1-kepler_elements.ecc*math.cos(E_check))

# ensure we land in the correct quadrant:
nu_check = math.atan2(nu_check_sin, nu_check_cos)
if nu_check < 0:
    nu_check = nu_check + 2*math.pi
print("nu propagated: " + str(math.degrees(nu_check)) + " deg")

nu propagated: 64.99999999999997 deg


##### Starting at Vo, what is the true anomaly after 2700 seconds?

In [88]:
# Propagate nu given delta t
import math

def compute_propagate_nu_given_delta_t(kepler_elements: KeplerianElements, time_delta: float) -> tuple[float, int, float]:
    E0 = compute_eccentric_anomaly(kepler_elements.ta, kepler_elements.ecc)
    M0 = compute_mean_anomaly(E0, kepler_elements.ecc)
    n = compute_mean_motion(kepler_elements.mu_earth, kepler_elements.a)
    M_propagated = M0 + n*time_delta
    E_k = M_propagated
    E_k_1 = 100000000000 # just to make sure our diff is large enough for first iteration
    while (abs(E_k-E_k_1) != 0):
        E_k = E_k_1
        E_k_1=E_k + (M_propagated-(E_k-kepler_elements.ecc*math.sin(E_k)))/(1-kepler_elements.ecc*math.cos(E_k))
    E_propagated = E_k_1%(2*math.pi)
    k = math.floor((E_k_1-E0)/(2*math.pi))
    cos_nu = (kepler_elements.ecc-math.cos(E_propagated))/(kepler_elements.ecc*math.cos(E_propagated)-1)
    sin_nu = (math.sin(E_propagated)*math.sqrt(1-math.pow(kepler_elements.ecc, 2)))/(1-kepler_elements.ecc*math.cos(E_propagated))
    nu_propagated = math.atan2(sin_nu, cos_nu)
    # ensure we land in the correct quadrant:
    nu_propagated = math.atan2(math.sin(nu_propagated), math.cos(nu_propagated))
    if nu_propagated < 0:
        nu_propagated = nu_propagated + 2*math.pi
    return E_propagated, k, nu_propagated

In [89]:
# calculate nu after 2700 seconds:
E_propagated, k, nu_propagated_2700 = compute_propagate_nu_given_delta_t(kepler_elements, 2700)
print("Propagated E (2700): " + str(E_propagated) + " rad")
print("Propagated nu (2700): " + str(nu_propagated_2700) + " rad")
print("k (2700): " + str(k))


Propagated E (2700): 3.546268977283855 rad
Propagated nu (2700): 3.5423496855494134 rad
k (2700): 0


##### Starting at nu_0, what is the true anomaly after exactly two orbit periods?  
The true anomaly is the same as it was at nu_0 which is `0.533708002792791 rad` but here are the computations anyway to prove it.

In [90]:
# calculate nu after 2 orbits
two_orbit_period_time_diff = kepler_elements.period * 2
E_propagated, k, nu_propagated_2_orbits = compute_propagate_nu_given_delta_t(kepler_elements, two_orbit_period_time_diff)
print("Propagated E (2 orbits): " + str(E_propagated) + " rad")
print("Propagated nu (2 orbits): " + str(nu_propagated_2_orbits) + " rad")
print("k (2 orbits): " + str(k))

Propagated E (2 orbits): 0.5286424011417861 rad
Propagated nu (2 orbits): 0.5337080027927918 rad
k (2 orbits): 2


##### What is the true anomaly after 15000 seconds?

In [91]:
# calculate nu after 15000 seconds:
E_propagated, k, nu_propagated_15000 = compute_propagate_nu_given_delta_t(kepler_elements, 15000)
print("Propagated E (15000): " + str(E_propagated) + " rad")
print("Propagated nu (15000): " + str(nu_propagated_15000) + " rad")
print("k (15000): " + str(k))

Propagated E (15000): 4.7617259166585875 rad
Propagated nu (15000): 4.751735454771401 rad
k (15000): 2


# Problem 2:  
##### Compute Keplerian elements

In [92]:
pos_ = Vector3(572461.711228, -1015437.194396, 7707337.871302)  # meters
vel_ = Vector3(-6195.262945, -3575.889650, -5.423283) # m/s

kepler_elements = KeplerianElements(pos_, vel_)
print(f"{'Semi-Major Axis (a):':<30} {kepler_elements.a:16.8f} m")
print(f"{'Eccentricity (e):':<30} {kepler_elements.ecc:16.8f}")
print(f"{'Inclination (inc):':<30} {kepler_elements.inc_deg:16.8f} deg")
print(f"{'RAAN (raan):':<30} {kepler_elements.raan_deg:16.8f} deg")
print(f"{'Argument of Perigee (wp):':<30} {kepler_elements.argp_deg:16.8f} deg")
print(f"{'True Anomaly (nu):':<30} {kepler_elements.ta_deg:16.8f} deg")

Semi-Major Axis (a):           7800000.00120126 m
Eccentricity (e):                    0.00100000
Inclination (inc):                  98.60000000 deg
RAAN (raan):                        30.00000000 deg
Argument of Perigee (wp):           40.00000696 deg
True Anomaly (nu):                  50.08784582 deg


##### Find perifocal position and velocity
calculate r using:  
`r = p(1+e*cos(nu))^-1`  
calculate semi-latus rectum using:  
`p = a(1 - e^2)`  
Then use that in these equations:  
`pos_x = r*cos(nu)`  
`pos_y = r*sin(nu)`  
`vel_x = -sqrt(mu/p)*sin(nu)`  
`vel_y = sqrt(mu/p)*(e+cos(nu))`  

In [93]:
# Compute Perifocal Coordinates
import math

def compute_perifocal_coordinates(kepler_elements: KeplerianElements) -> tuple[Vector3, Vector3]:
    p = kepler_elements.a*(1-math.pow(kepler_elements.ecc,2))
    r = p*(1/(1+kepler_elements.ecc*math.cos(kepler_elements.ta)))
    perifocal_pos_x = r*math.cos(kepler_elements.ta)
    perifocal_pos_y = r*math.sin(kepler_elements.ta)
    perifocal_vel_x = -math.sqrt(kepler_elements.mu_earth/p)*math.sin(kepler_elements.ta)
    perifocal_vel_y = math.sqrt(kepler_elements.mu_earth/p)*(kepler_elements.ecc + math.cos(kepler_elements.ta))
    perifocal_pos = Vector3(perifocal_pos_x, perifocal_pos_y, 0)
    perifocal_vel = Vector3(perifocal_vel_x, perifocal_vel_y, 0)
    return perifocal_pos, perifocal_vel

In [94]:
perifocal_pos, perifocal_vel = compute_perifocal_coordinates(kepler_elements)
print("Perifocal Pos: " + str(perifocal_pos))
print("Perifocal Vel: " + str(perifocal_vel))

Perifocal Pos: Vector3(x=5001362.438709829, y=5978984.522950811, z=0)
Perifocal Vel: Vector3(x=-5483.194150338307, y=4593.7872249740285, z=0)


##### Find f, g, f', g' for delta nu = 33°
$\Large f=\frac{x \dot{y}_0 - \dot{x}_0 y}{h}$  
$\Large g=\frac{x_0 y - x y_0}{h}$
  
$\Large \dot{f} = \frac{\dot{x} \dot{y}_0 - \dot{x}_0 \dot{y}}{h}$  
$\Large \dot{g} = \frac{x_0 \dot{y} - \dot{x} y_0}{h}$

In [97]:
# compute f and g
def compute_f_g_f_dot_g_dot(pos_0, vel_0, pos, vel:Vector3) -> tuple[float, float, float, float]:
    h = compute_angular_momentum_vector(pos_0, vel_0)
    f = (pos.x*vel_0.y - vel_0.x*pos.y)/h.magnitude()
    g = (pos_0.x*pos.y - pos.x*pos_0.y)/h.magnitude()
    f_dot = (vel.x*vel_0.y - vel_0.x*vel.y)/h.magnitude()
    g_dot = (pos_0.x*vel.y - vel.x*pos_0.y)/h.magnitude()
    return f, g, f_dot, g_dot

In [98]:
import copy
pos_ = Vector3(572461.711228, -1015437.194396, 7707337.871302)  # meters
vel_ = Vector3(-6195.262945, -3575.889650, -5.423283) # m/s

kepler_elements = KeplerianElements(pos_, vel_)
perifocal_pos, perifocal_vel = compute_perifocal_coordinates(kepler_elements)

r_0 = perifocal_pos
v_0 = perifocal_vel

_, time_delta = compute_time_delta_after_angle(math.radians(math.degrees(kepler_elements.ta)+33), kepler_elements)

_, _, nu_propagated = compute_propagate_nu_given_delta_t(kepler_elements, time_delta)

kepler_elements_propagated_nu = copy.deepcopy(kepler_elements)
kepler_elements_propagated_nu.ta = nu_propagated

r_0, v_0 = compute_perifocal_coordinates(kepler_elements)
r, v = compute_perifocal_coordinates(kepler_elements_propagated_nu)

f, g, f_dot, g_dot = compute_f_g_f_dot_g_dot(r_0, v_0, r, v)
print(f"{'f:':<30} {f:16.8f}")
print(f"{'g:':<30} {g:16.8f}")
print(f"{'f_dot:':<30} {f_dot:16.8f}")
print(f"{'g_dot:':<30} {g_dot:16.8f}")


f:                                   0.83868998
g:                                 593.81382837
f_dot:                              -0.00049936
g_dot:                               0.83877401


##### find r_ and v_ using

$\vec{r} = f \,\vec{r}_0 + g \,\vec{v}_0$
  
$\vec{v} = \dot{f} \,\vec{r}_0 + \dot{g} \,\vec{v}_0$

In [101]:
r_ = f*r_0 + g*v_0
v_ = f_dot*r_0 + g_dot*v_0

print("r_ = " + str(r_))
print("v_ = " + str(v_))

r_ = Vector3(x=938596.0595633695, y=7742368.795881752, z=0.0)
v_ = Vector3(x=-7096.655979287696, y=867.4658519783247, z=0.0)


# Problem 3:  
##### Derivation 1: using energy equation and ellipse equation, find an expression for the orbit speed as a function of true anomaly

##### Derivation 2: using the above show that 
$\Large
v_{\text{perigee}} = \sqrt{ \frac{\mu}{a} \left( \frac{1+e}{1-e} \right) }
$

$\Large
v_{\text{apogee}} = \sqrt{ \frac{\mu}{a} \left( \frac{1-e}{1+e} \right) }
$